
# CONNECT 4: Inteligência Artificial com Monte Carlo Tree Search e Árvores de Decisão (ID3)

**Desenvolvido por:** Cauã Pinheiro Souza e Matheus Gonçalves Guerra

---

## Introdução

Connect 4 é um jogo clássico de estratégia para dois jogadores, cujo objetivo é posicionar quatro peças consecutivas da mesma cor em linha — podendo ser horizontal, vertical ou diagonal. O jogo é disputado em um tabuleiro composto por sete colunas e seis linhas, onde os jogadores se alternam para inserir suas peças nas colunas disponíveis. O objetivo principal é formar uma sequência vencedora, enquanto simultaneamente bloqueia as jogadas do adversário.

Neste trabalho, desenvolvemos uma aplicação em Python que simula o jogo Connect 4 e explora técnicas de inteligência artificial para aprimorar a experiência e o desafio oferecido pelo sistema. A aplicação oferece três modos de jogo distintos:

- **Jogador vs Jogador:** dois usuários humanos competem entre si no mesmo dispositivo.
- **Jogador vs Inteligência Artificial:** um jogador humano enfrenta uma IA que decide suas jogadas.
- **Inteligência Artificial vs Inteligência Artificial:** duas diferentes estratégias de IA competem uma contra a outra, possibilitando análise comparativa.

Para o desenvolvimento da inteligência artificial, utilizamos duas abordagens principais:

1. **Monte Carlo Tree Search (MCTS):** uma técnica de simulação que avalia as possíveis jogadas por meio de centenas ou milhares de partidas simuladas, buscando selecionar o melhor movimento baseado em probabilidades estatísticas.
2. **Árvores de Decisão (ID3):** um algoritmo de aprendizado supervisionado que cria modelos de decisão a partir de dados de jogos anteriores, permitindo que a IA aprenda padrões e estratégias eficazes para melhorar sua performance.
3. **Alpha-Beta Pruning:** uma técnica de otimização usada em algoritmos de busca minimax para jogos. Ela elimina ramos da árvore de decisão que não influenciarão o resultado final, reduzindo significativamente o número de nós avaliados. Com isso, a IA consegue analisar posições mais profundamente em menos tempo, mantendo a decisão ótima.
4. **Heurística:** uma função de avaliação que estima o valor de uma posição de jogo sem simular até o final. Utiliza critérios como alinhamentos, ameaças e controle do centro para atribuir uma pontuação a cada estado. Isso permite que a IA tome decisões rápidas e eficazes, mesmo sem explorar todas as possibilidades até o fim do jogo.

Este notebook apresenta a implementação do jogo, descreve a estrutura dos códigos e detalha como as técnicas de IA foram aplicadas para criar adversários cada vez mais inteligentes, além de discutir exemplos de uso e testes.



## Estrutura do Projeto

O projeto está organizado em diferentes pastas, cada uma com uma responsabilidade específica:

- `main.py` – Arquivo principal para execução do jogo.
- `ai_alg/` – Algoritmos de Inteligência Artificial (Monte Carlo, heurísticas, Alpha-Beta).
- `game_structure/` – Estrutura central do jogo: regras, tabuleiro, interface gráfica, etc.
- `training/` – Scripts para Árvores de Decisão, geração/análise de datasets e ID3.
- `requirements.txt` – Lista de dependências Python.


### main.py

**Função:** Implementa funcionalidades do módulo `main.py`.

**Funções e Classes Presentes:**
- **Função `main()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
from game_structure.Board import Board
from game_structure.interface import Interface


def main() -> None:
    board = Board()
    interface = Interface()
    interface.starting_game(board)


if __name__ == "__main__":
    main()


### alpha_beta.py

**Função:** Implementa funcionalidades do módulo `alpha_beta.py`.

**Funções e Classes Presentes:**
- **Função `alpha_beta()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `evaluate_position()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `generate_children()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import numpy as np
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import heuristic as h


def alpha_beta(board: np.ndarray) -> int:
    """Usando o algoritmo de alpha-beta, escolhe a melhor jogada até uma certa profundidade"""
    max_depth = 5
    best_col = -1
    best_score = float('-inf')

    for new_board, col in generate_children(board, s.SECOND_PLAYER_PIECE):
        if game.winning_move(new_board, s.SECOND_PLAYER_PIECE):
            return col
        score = evaluate_position(
            new_board, current_depth=1,
            alpha=float('-inf'), beta=float('inf'),
            depth_limit=max_depth, is_maximizing=False
        )
        if score > best_score:
            best_score = score
            best_col = col

    return best_col


def evaluate_position(board: np.ndarray,current_depth: int,alpha: float,beta: float,depth_limit: int,is_maximizing: bool) -> float:
    """Usa minimax e alpha-beta para avaliar o tabuleiro"""
    if (current_depth == depth_limit or game.winning_move(board, s.FIRST_PLAYER_PIECE) or game.winning_move(board, s.SECOND_PLAYER_PIECE) or game.is_game_tied(board)):
        return h.calculate_board_score(board, s.SECOND_PLAYER_PIECE, s.FIRST_PLAYER_PIECE)

    if is_maximizing:
        max_eval = float('-inf')
        for child_board, _ in generate_children(board, s.SECOND_PLAYER_PIECE):
            evaluation = evaluate_position(child_board, current_depth + 1, alpha, beta, depth_limit, False)
            max_eval = max(max_eval, evaluation)
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = float('inf')
        for child_board, _ in generate_children(board, s.FIRST_PLAYER_PIECE):
            evaluation = evaluate_position(child_board, current_depth + 1, alpha, beta, depth_limit, True)
            min_eval = min(min_eval, evaluation)
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return min_eval


def generate_children(board: np.ndarray, piece: int):
    """Gera tabuleiros filhos para uma determinada peça."""
    children = []
    available = game.available_moves(board)
    if available == -1:
        return children
    for col in available:
        new_board = game.simulate_move(board, piece, col)
        children.append((new_board, col))
    return children


### basic_heuristic.py

**Função:** Implementa funcionalidades do módulo `basic_heuristic.py`.

**Funções e Classes Presentes:**
- **Função `evaluate_best_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `adversarial_lookahead()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import numpy as np
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import heuristic as h


def evaluate_best_move(board: np.ndarray, player_piece: int, enemy_piece: int) -> int:
    """Avalia todas as possíveis jogadas e retorna aquela com a maior pontuação, utilizando a função heurística"""
    top_score = float('-inf')
    chosen_column = -1
    for column in game.available_moves(board):
        future_board = game.simulate_move(board, player_piece, column)
        evaluation = h.calculate_board_score(future_board, player_piece, enemy_piece)
        if evaluation > top_score:
            top_score = evaluation
            chosen_column = column
    return chosen_column


def adversarial_lookahead(board: np.ndarray, player_piece: int, enemy_piece: int) -> int:
    """Considera a próxima jogada e a melhor repostas possível do adversário a esta resposta, escolhendo assim a jogada com melhor vantagem"""
    optimal_column = -1
    highest_evaluation = float('-inf')
    suggested_counter = 0

    legal_columns = game.available_moves(board)
    if len(legal_columns) == 1:
        return legal_columns[0]

    for column in legal_columns:
        player_future = game.simulate_move(board, player_piece, column)
        if game.winning_move(player_future, s.SECOND_PLAYER_PIECE):
            return column
        predicted_opponent = evaluate_best_move(player_future, enemy_piece, player_piece)
        opponent_future = game.simulate_move(player_future, enemy_piece, predicted_opponent)
        evaluation = h.calculate_board_score(opponent_future, player_piece, enemy_piece)
        if evaluation > highest_evaluation:
            highest_evaluation = evaluation
            optimal_column = column
            suggested_counter = predicted_opponent + 1

    print("Possivel jogada: coluna no", suggested_counter + 1)
    return optimal_column


### heuristic.py

**Função:** Implementa funcionalidades do módulo `heuristic.py`.

**Funções e Classes Presentes:**
- **Função `calculate_board_score()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `weights()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

### Árvore de Decisão - 300 jogos
![Árvore de Decisão - 300 jogos](https://github.com/mtsguerra/Connect4/raw/main/training/data/connect4_300games_tree.png)

In [ ]:
from game_structure import style as s
import numpy as np

def calculate_board_score(board: np.ndarray, piece: int, opponent_piece: int) -> int:
    score = 0

    # Verifica horizontal
    for col in range(s.COLUMNS - 3):
        for r in range(s.ROWS):
            segment = [board[r][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica vertical
    for col in range(s.COLUMNS):
        for r in range(s.ROWS - 3):
            segment = [board[r + i][col] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica diagonal ascendente
    for col in range(s.COLUMNS - 3):
        for r in range(s.ROWS - 3):
            segment = [board[r + i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    # Verifica diagonal descendente
    for col in range(s.COLUMNS - 3):
        for r in range(3, s.ROWS):
            segment = [board[r - i][col + i] for i in range(4)]
            score += weights(segment, piece, opponent_piece)

    return score


def weights(segment: list, piece: int, opponent_piece: int) -> int:
    if piece in segment and opponent_piece in segment: return 0
    if segment.count(piece) == 1: return 1
    if segment.count(piece) == 2: return 10
    if segment.count(piece) == 3: return 50
    if segment.count(piece) == 4: return 1000
    if segment.count(opponent_piece) == 1: return -1
    if segment.count(opponent_piece) == 2: return -10
    if segment.count(opponent_piece) == 3: return -50
    if segment.count(opponent_piece) == 4: return -2000
    return 0


### monte_carlo.py

**Função:** Implementa funcionalidades do módulo `monte_carlo.py`.

**Funções e Classes Presentes:**
- **Função `mcts()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `__init__()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `__str__()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `add_children()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `select_children()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `ucb()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `score()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `__init__()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `start()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `search()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `select()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `best_child()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `back_propagation()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `expand()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `rollout()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `best_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Classe `Node`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.
- **Classe `monte_carlo`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.

In [ ]:
import time, math, numpy as np, random, itertools
from game_structure import style as s
from game_structure import game_engine as game
from math import sqrt, log
from ai_alg import heuristic as h

class Node:
    """Representa um nó na árvore de busca Monte Carlo, onde cada nó representa uma configuração diferente do tabuleiro"""
    
    def __init__(self, board, last_player, parent=None) -> None:
        self.board = board  # Estado atual do tabuleiro
        self.parent = parent  # Nó pai na árvore
        self.children = []  # Lista de nós filhos
        self.visits = 0  # Número de vezes que este nó foi visitado
        self.wins = 0  # Número de vitórias alcançadas a partir deste nó
        # Determina o jogador atual com base em que jogou por ultimo
        self.current_player = 1 if last_player == 2 else 2

    def __str__(self) -> str:
        """Representação em string do nó para depuração"""
        
        string = "Vitórias: " + str(self.wins) + '\n'
        string += "Total de visitas: " + str(self.visits) + '\n'
        string += "Pontuação UCB: " + str(self.ucb()) + '\n'
        string += "Probabilidade de vitória: " + str(self.score()) + '\n'
        return string

    def add_children(self) -> None:
        """Gera todos possíveis movimentos no tabuleiro atual, adicionando-os como filhos deste nó"""
        
        if (len(self.children) != 0) or (game.available_moves(self.board) == -1):
            return
            
        for col in game.available_moves(self.board):
            # Cria uma cópia do tabuleiro com a nova jogada aplicada
            if self.current_player == s.FIRST_PLAYER_PIECE:
                copy_board = game.simulate_move(self.board, s.SECOND_PLAYER_PIECE, col)
            else:
                copy_board = game.simulate_move(self.board, s.FIRST_PLAYER_PIECE, col)
                
            # Adiciona o novo nó e a coluna que o gerou à lista de filhos
            self.children.append((Node(board=copy_board, last_player=self.current_player, parent=self), col))

    def select_children(self):
        """Seleciona aleatoriamente no máximo 4 filhos para apurar a busca, melhorando assim a sua eficiência"""
        
        if (len(self.children) > 4):
            return random.sample(self.children, 4)
        return self.children

    def ucb(self) -> float:
        """Cacula o UCB (Upper Confidence Bound) para escolher os nós, equilibrando assim as escolhas aprofundadas e as não"""
        
        if self.visits == 0:
            return float('inf')  # Nós não visitados têm prioridade máxima
            
        # Fórmula UCB: wins/visits + C * sqrt(ln(parent_visits)/visits)
        exploitation = self.wins / self.visits
        exploration = sqrt(2) * sqrt(log(self.parent.visits) / self.visits)
        return exploitation + exploration
        
    def score(self) -> float:
        """Calcula a taxa de vitórias para este nó"""
        if self.visits == 0:
            return 0
        return self.wins / self.visits


class monte_carlo:
    """Implementação do algoritmo de Monte Carlo TS, usando UCB1 para escolher os nós e simulando usando heurística"""
    
    def __init__(self, root: Node) -> None:
        self.root = root

    def start(self, max_time: int):
        """Inicia a busca realizando simulações iniciais nos filhos da raiz, executando em seguida o algoritmo mcts"""
        
        self.root.add_children()   
        for child in self.root.children:
            if game.winning_move(child[0].board, s.SECOND_PLAYER_PIECE):
                return child[1]  
            for _ in range(6):
                result = self.rollout(child[0])
                self.back_propagation(child[0], result)
                
        return self.search(max_time)

    def search(self, max_time: int) -> int:
        """Executa o algoritmo princiapl do mcts até o limite de tempo"""
        
        start_time = time.time()
        while time.time() - start_time < max_time:
            selected_node = self.select(self.root)
            
            # Analisa se o nó foi ou não visitado, mudando ou não de onde partir
            if selected_node.visits == 0:
                result = self.rollout(selected_node)
                self.back_propagation(selected_node, result)
            else:
                selected_children = self.expand(selected_node)
                for child in selected_children:
                    result = self.rollout(child[0])
                    self.back_propagation(child[0], result)
                                   
        return self.best_move()

    def select(self, node: Node) -> Node:
        """Seleciona o nó com maiores chances, utilizando UCB para equilibrar"""
        
        if node.children == []:
            return node
        else:
            node = self.best_child(node)
            return self.select(node)

    def best_child(self, node: Node) -> Node:
        best_child = None
        best_score = float('-inf')
        
        for (child, _) in node.children:
            ucb = child.ucb()
            if ucb > best_score:
                best_child = child
                best_score = ucb
                
        return best_child

    def back_propagation(self, node: Node, result: int) -> None:
        """Atualiza os dados para todos os nós da árvore, incrementando visit count e possívelmente win count"""
        
        while node:
            node.visits += 1
            if node.current_player == result:
                node.wins += 1
            node = node.parent

    def expand(self, node: Node) -> Node:
        """Adiciona todos os possíveis nós filhos e seleciona alguns"""
        
        node.add_children()
        return node.select_children()

    def rollout(self, node: Node) -> int:
        """Simula uma possível partida a partir do nó atual, utilizando de jogadas guiadas por heurística, retornando assim o vencedor da simulação"""
        
        board = node.board.copy()
        max_depth = 6
        players = itertools.cycle([s.SECOND_PLAYER_PIECE, s.FIRST_PLAYER_PIECE])
        current_player = next(players)

        for _ in range(max_depth):
            if game.winning_move(board, s.SECOND_PLAYER_PIECE) or game.winning_move(board, s.FIRST_PLAYER_PIECE):
                break
            if game.is_game_tied(board):
                return 0   
            current_player = next(players)
            moves = game.available_moves(board) 
            if moves == -1:
                return 0 
            # Usa heurística para escolher a melhor jogada (não aleatória)
            best_move = max(
                moves,
                key=lambda col: h.calculate_board_score(
                    game.simulate_move(board, current_player, col), 
                    s.SECOND_PLAYER_PIECE, 
                    s.FIRST_PLAYER_PIECE
                )
            )
            
            board = game.simulate_move(board, current_player, best_move)

        return current_player

    def best_move(self) -> int:
        """Seleciona as melhores opções, baseado na taxa de vitória dos filhos da raíz, possívelmente com a mesma pontuação, "desempatando" aleatoriamente"""
        
        max_score = float('-inf')
        scores = {}  # Armazena os pares de colunas e suas respectivas pontuações
        columns = []  # Armazena as colunas com a melhor pontuação
        
        # Calcula as pontuações possíveis
        for (child, col) in self.root.children:
            score = child.score()
            print(f"Coluna: {col}")
            print(child)
            
            if score > max_score:
                max_score = score
                
            scores[col] = score
            
        # Armazena todos os movimentos que resultam com a melhor pontuação
        for col, score in scores.items():
            if score == max_score:
                columns.append(col)
                
        # Escolhe aleatoriamente entre os melhores movimentos
        return random.choice(columns)


def mcts(board: np.ndarray) -> int:
    """Função que executa a busca mcts e retorna a melhor coluna para se jogar"""
    
    root = Node(board=board, last_player=s.SECOND_PLAYER_PIECE)
    mc = monte_carlo(root)
    column = mc.start(3)  # Executa o MCTS por 3 segundos
    print(column + 1)
    return column


### Board.py

**Função:** Implementa funcionalidades do módulo `Board.py`.

**Funções e Classes Presentes:**
- **Função `get_board()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `print_board()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Classe `Board`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.

### Distribuição dos Movimentos - 3500 jogos, 1.5s
![Distribuição dos Movimentos - 3500 jogos, 1.5s](https://github.com/mtsguerra/Connect4/raw/main/training/data/connect4_3500games1point5sec_dataset.png)

In [ ]:
import numpy as np
from dataclasses import dataclass, field
from game_structure import style as s

@dataclass
class Board:
    rows: int = s.ROWS
    columns: int = s.COLUMNS
    board: np.ndarray = field(default_factory=lambda: np.zeros((s.ROWS, s.COLUMNS)))

    def get_board(self):
        return self.board

    def print_board(self):
        print(np.flip(self.board, 0), "\n")


### ai_game.py

**Função:** Implementa funcionalidades do módulo `ai_game.py`.

**Funções e Classes Presentes:**
- **Função `run_ai_vs_ai_game()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_ai_types()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_ai_column_for_type()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import pygame
import numpy as np
from game_structure import Board
from game_structure import style as s
from game_structure import game_engine as game
from ai_alg import basic_heuristic as b, alpha_beta as a, monte_carlo as m, heuristic as h
from training.decision_tree_player import decision_tree_move  # <-- Mantenha este import
import itertools
import time

def run_ai_vs_ai_game(interface, brd, game_mode):
    """Simula uma partida entre duas IA, podendo ou não serem diferentes"""
    
    board = brd.get_board()
    game_over = False
    turns = itertools.cycle([1, 2]) 
    turn = next(turns)
    
    # Define qual IA cada jogador será
    ai_types = get_ai_types(game_mode)
    
    interface.draw_board()
    pygame.display.update()
    
    pygame.time.wait(1000)
    
    move_count = 0
    
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                interface.quit()
                
        # Pequeno atraso entre as jogadas para as partidas poderem serem assistidas
        pygame.time.wait(500)
        
        ai_type = ai_types[turn - 1]
        col = get_ai_column_for_type(board, turn, ai_type)
        
        row = game.get_next_open_row(board, col)
        if row != -1:
            game.drop_piece(board, row, col, turn)
            interface.draw_new_piece(row + 1, col + 2, turn)
            pygame.display.update()
            brd.print_board()
            move_count += 1
            print(f"Jogada {move_count}: Jogador {turn} ({ai_type}) colocou na coluna {col}")
            
            if game.winning_move(board, turn):
                font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
                interface.show_winner(font, turn)
                game_over = True
                break  
            if game.is_game_tied(board):
                font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
                interface.show_draw(font)
                game_over = True
                break
                
        turn = next(turns)
    
    pygame.time.wait(10000)
    return

def get_ai_types(game_mode):
    """Retorna as IAs escolhidas"""
    
    ai_mapping = {
        6: ["Easy", "Easy"],           # Fácil x Fácil
        7: ["Easy", "Hard"],           # Fácil x Difícil
        8: ["Medium", "Medium"],       # Médio x Médio
        9: ["Medium", "Challenge"],    # Médio x Desafio
        10: ["Hard", "Challenge"],     # AlphaBeta x Desafio
        11: ["Easy", "Medium"],        # Fácil x Médio
        12: ["Easy", "Challenge"],     # Fácil x Desafio
        13: ["Medium", "Hard"],        # Médio x AlphaBeta
        14: ["Hard", "Hard"],          # AlphaBeta x AlphaBeta
        15: ["Challenge", "Challenge"] # Desafio x Desafio
    }
    return ai_mapping.get(game_mode, ["Easy", "Easy"])

def get_ai_column_for_type(board, player, ai_type):
    """Obtém a melhor jogada de acordo com a respectiva IA"""
    
    opponent = 1 if player == 2 else 2
    
    if ai_type == "Easy":
        # Usa heurística básica (A*)
        return b.evaluate_best_move(board, player, opponent)
    elif ai_type == "Medium":
        # Usa Monte Carlo Tree Search (MCTS)
        return m.mcts(board)
    elif ai_type == "Hard":
        # Usa poda Alpha-Beta
        return a.alpha_beta(board)
    elif ai_type == "Challenge":
        # Usa agente Decision Tree
        return decision_tree_move(board)
    else:
        # Padrão para heurística básica
        return b.evaluate_best_move(board, player, opponent)


### game_engine.py

**Função:** Implementa funcionalidades do módulo `game_engine.py`.

**Funções e Classes Presentes:**
- **Função `first_player_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_human_column()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `available_moves()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `ai_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_ai_column()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `simulate_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `make_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_next_open_row()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `drop_piece()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `is_game_tied()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `is_valid()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `winning_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
from game_structure import style as s
import numpy as np
import math
import pygame
from game_structure import Board
from ai_alg import basic_heuristic as b, alpha_beta as a, monte_carlo as m
from training.decision_tree_player import decision_tree_move

def first_player_move(bd: Board, interface: any, board: np.ndarray, turn: int, event: any) -> bool:
    """Define a coluna jogada pelo jogador 1"""
    
    col = get_human_column(interface, event)
    if not is_valid(board, col): return False

    pygame.draw.rect(interface.screen, s.BACKGROUND_COLOR, (0, 0, interface.width, interface.pixels - 14))
    make_move(bd, interface, board, turn, col)
    return True

def get_human_column(interface: any, event: any):
    """Obtém a coluna selecionada pelo mouse"""
    
    posx = event.pos[0]
    col = int(math.floor(posx / interface.pixels)) - 2
    return col

def available_moves(board: np.ndarray) -> list | int:
    """Retorna uma lista de colunas disponíveis para jogar, ou -1 se não houver"""
    
    avaiable_moves = []
    for i in range(s.COLUMNS):
        if (board[5][i]) == 0:
            avaiable_moves.append(i)
    return avaiable_moves if len(avaiable_moves) > 0 else -1

def ai_move(bd: Board, interface: any, game_mode: int, board: np.ndarray, turn: int) -> int:
    """Define a coluna jogada pelo jogador 2"""
    
    ai_column = get_ai_column(board, game_mode, turn)
    game_over = make_move(bd, interface, board, turn, ai_column)
    return game_over

def get_ai_column(board: Board, game_mode: int, player: int = 2) -> int:
    """Seleciona o algoritmo de IA escolhido para jogar"""
    
    opponent = 1 if player == 2 else 2

    if game_mode == 2:
        return b.evaluate_best_move(board, player, opponent)
    elif game_mode == 3:
        return m.mcts(board)
    elif game_mode == 4:
        return a.alpha_beta(board)
    elif game_mode == 5:
        return decision_tree_move(board)
    return 0

def simulate_move(board: np.ndarray, piece: int, col: int) -> np.ndarray:
    """Simula uma jogada em uma cópia do tabuleiro"""
    
    board_copy = board.copy()
    row = get_next_open_row(board_copy, col)
    drop_piece(board_copy, row, col, piece)
    return board_copy

def make_move(bd: Board, interface: any, board: np.ndarray, turn: int, move: int):
    """Executa a jogada e verifica se ela resulta em vitória ou empate"""

    row = get_next_open_row(board, move)
    drop_piece(board, row, move, turn) 
    interface.draw_new_piece(row + 1, move + 2, turn)
    pygame.display.update()
    bd.print_board()

    return winning_move(board, turn) or is_game_tied(board)

def get_next_open_row(board: np.ndarray, col: int) -> int:
    """Retorna a linha disponível para colocar a peça na coluna escolhida"""
    
    for row in range(s.ROWS):
        if board[row, col] == 0:
            return row
    return -1

def drop_piece(board: np.ndarray, row: int, col: int, piece: int) -> None:
    """Insere a peça no tabuleiro na posição escolhida"""
    
    board[row, col] = piece

def is_game_tied(board: np.ndarray) -> bool:
    """Verifica se o jogo empatou"""
    
    if winning_move(board, s.SECOND_PLAYER_PIECE) or winning_move(board, s.FIRST_PLAYER_PIECE): return False
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0: return False
    return True

def is_valid(board: np.ndarray, col: int) -> bool:
    """Analisa se a coluna escolhida é válida"""
    
    if not 0 <= col < s.COLUMNS: return False
    row = get_next_open_row(board, col)
    return 0 <= row <= 5

def winning_move(board: np.ndarray, piece: int) -> bool:
    """Analisa se a peça colocada resultou em vitória"""
    
    rows, cols = board.shape
    for row in range(rows):
        for col in range(cols):
            if int(board[row, col]) == piece:
                # Checa horizontalmente
                if col + 3 < cols and all(board[row, col + i] == piece for i in range(4)):
                    return True
                # Checa verticalmente
                if row + 3 < rows and all(board[row + i, col] == piece for i in range(4)):
                    return True
                # Checa diagonal ascendente
                if row - 3 >= 0 and col + 3 < cols and all(board[row - i, col + i] == piece for i in range(4)):
                    return True
                # Checa diagonal descendente
                if row + 3 < rows and col + 3 < cols and all(board[row + i, col + i] == piece for i in range(4)):
                    return True
    return False


### interface.py

**Função:** Implementa funcionalidades do módulo `interface.py`.

**Funções e Classes Presentes:**
- **Função `starting_game()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `play()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_menu()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `render_alternating_colors_text()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `choose_option()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_combinations()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `choose_AI_combination()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_difficulties()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `choose_AI_difficulty()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_board()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_new_piece()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `draw_button()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `show_winner()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `show_draw()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `quit()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Classe `Interface`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.

In [ ]:
import pygame
import itertools
import sys
from game_structure import style as s
from game_structure import Board
from game_structure import game_engine as game
from dataclasses import dataclass
from game_structure.ai_game import run_ai_vs_ai_game

@dataclass
class Interface:
    rows: int = s.ROWS
    columns: int = s.COLUMNS
    pixels: int = s.SQUARE_SIZE
    width: int = s.WIDTH
    height: int = s.HEIGHT
    rad: float = s.RADIUS_PIECE
    size: tuple = (width, height)
    screen: any = pygame.display.set_mode(size)
    pygame.display.set_caption("Connect4")

    def starting_game(self, brd: Board):
        """Inicia o programa para rodar o jogo"""
        
        pygame.init()
        self.draw_menu()
        game_mode = self.choose_option()
        brd.print_board()
        self.draw_board()
        pygame.display.update()
        
        if game_mode >= 6 and game_mode <= 15:  # Modos IA vs IA
            run_ai_vs_ai_game(self, brd, game_mode)
        else:
            self.play(brd, game_mode)

    def play(self, brd: Board, game_mode: int):
        board = brd.get_board()
        game_over = False
        # Mario !!
        font = pygame.font.SysFont('Connect4-main/fonts/SuperMario256.ttf', 50)
        turns = itertools.cycle([1, 2])
        turn = next(turns)

        while not game_over:
            for event in pygame.event.get():
                if event.type == pygame.QUIT: quit()

                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(self.screen, s.BACKGROUND_COLOR, (0, 0, self.width, self.pixels - 14))
                    posx = event.pos[0]
                    pygame.draw.circle(self.screen, s.PIECES_COLORS[turn], (posx, int(self.pixels / 2) - 7), self.rad)
                pygame.display.update()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    if turn == 1 or (turn == 2 and game_mode == 1): 
                        if not game.first_player_move(brd, self, board, turn, event): continue  # verifica se a coluna é valida
                        if game.winning_move(board, turn):
                            game_over = True
                            break
                        turn = next(turns)

                if turn != 1 and game_mode != 1:
                    pygame.time.wait(15)
                    game_over = game.ai_move(brd, self, game_mode, board,turn)  # recebe jogada da IA e retorna se o jogo acabou
                    if game_over: break
                    turn = next(turns)

            if game.is_game_tied(board):
                self.show_draw(font)
                break

        if game.winning_move(board, turn):
            self.show_winner(font, turn)

        pygame.time.wait(10000)

    def draw_menu(self):
        """Desenha o menu e as opções do jogo"""
        
        self.screen.fill(s.BACKGROUND_COLOR)
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 80)

        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        pos = (560 - font.size("Connect 4")[0] // 2, 230 - font.get_height() // 2)

        self.render_alternating_colors_text("Connect 4", font, colors, pos)

        self.draw_button(self.height / 2, 350, 300, 50, "Single Player")
        self.draw_button(self.height / 2, 450, 300, 50, "Multiplayer")
        self.draw_button(self.height / 2, 550, 300, 50, "PC x PC")

    def render_alternating_colors_text(self, text: str, font, colors, pos):
        """Função para fazer os titulos ficarem igual ao super mario"""
        
        x, y = pos
        outline_color = s.BLACK
        outline_range = range(-3,4)

        for i, char in enumerate(text):
            color = colors[i % len(colors)]

            for ox in outline_range:
                for oy in outline_range:
                    if ox == 0 and oy == 0:
                        continue  # Pula o centro
                    outline_surface = font.render(char, True, outline_color)
                    self.screen.blit(outline_surface, (x + ox, y + oy))

            letter_surface = font.render(char, True, color)
            self.screen.blit(letter_surface, (x, y))
            x += letter_surface.get_width()  # Move x para o próximo caractere
        pygame.display.update()

    def choose_option(self) -> int:
        """Retorna o modo de jogo escolhido"""
        
        while True:
            game_mode = 0
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    quit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 350 <= mouse_y <= 400:
                        print("Player vs IA selecionado")
                        self.draw_difficulties()
                        game_mode = 2
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 450 <= mouse_y <= 500:
                        print("Player vs Player selecionado")
                        game_mode = 1
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 550 <= mouse_y <= 600:
                        print("IA vs IA selecionado")
                        game_mode = 3

            pygame.display.flip()

            if game_mode == 1:
                return game_mode

            if game_mode == 2:
                game_mode = self.choose_AI_difficulty()
                return game_mode

            if game_mode == 3:
                self.draw_combinations()
                game_mode = self.choose_AI_combination()
                return game_mode

    def draw_combinations(self):
        """Desenha as possíveis combinações entre as IAs"""
        
        self.screen.fill(s.BACKGROUND_COLOR)

        left_x = self.width / 4 - 200
        right_x = self.width * 3 / 4 - 200

        self.draw_button(left_x, 150, 400, 50, "Easy x Easy")  # A* x A*
        self.draw_button(left_x, 250, 400, 50, "Easy x Hard")  # A* x alpha beta
        self.draw_button(left_x, 350, 400, 50, "Medium x Medium")  # mcts x mcts
        self.draw_button(left_x, 450, 400, 50, "Medium x Challenge")  # mcts x decision tree
        self.draw_button(left_x, 550, 400, 50, "Hard x Challenge")  # alpha beta x decision tree

        self.draw_button(right_x, 150, 400, 50, "Easy x Medium")  # A* x mcts
        self.draw_button(right_x, 250, 400, 50, "Easy x Challenge")  # A* x decision tree
        self.draw_button(right_x, 350, 400, 50, "Medium x Hard")  # mcts x alpha beta
        self.draw_button(right_x, 450, 400, 50, "Hard x Hard")  # alpha beta x alpha beta
        self.draw_button(right_x, 550, 400, 50, "Challenge x Challenge")  # decision tree x decision tree

    def choose_AI_combination(self):
        """Retorna qual combinação foi escolhida"""
        
        left_x = self.width / 4 - 200
        right_x = self.width * 3 / 4 - 200
        game_mode = 0

        while True:
            for event in pygame.event.get():
                current_event = event.type
                if current_event == pygame.QUIT:
                    quit()
                elif current_event == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if left_x <= mouse_x <= left_x + 400:
                        if 150 <= mouse_y <= 200:
                            game_mode = 6
                            print("Easy x Easy")
                        elif 250 <= mouse_y <= 300:
                            game_mode = 7
                            print("Easy x Hard")
                        elif 350 <= mouse_y <= 400:
                            game_mode = 8
                            print("Medium x Medium")
                        elif 450 <= mouse_y <= 500:
                            game_mode = 9
                            print("Medium x Challenge")
                        elif 550 <= mouse_y <= 600:
                            game_mode = 10
                            print("Hard x Challenge")
                    elif right_x <= mouse_x <= right_x + 400:
                        if 150 <= mouse_y <= 200:
                            game_mode = 11
                            print("Easy x Medium")
                        elif 250 <= mouse_y <= 300:
                            game_mode = 12
                            print("Easy x Challenge")
                        elif 350 <= mouse_y <= 400:
                            game_mode = 13
                            print("Medium x Hard")
                        elif 450 <= mouse_y <= 500:
                            game_mode = 14
                            print("Hard x Hard")
                        elif 550 <= mouse_y <= 600:
                            game_mode = 15
                            print("Challenge x Challenge")
            pygame.display.flip()
            if game_mode != 0:
                return game_mode

    def draw_difficulties(self):
        """Desenha as dificuldades para single player"""
        
        self.screen.fill(s.BACKGROUND_COLOR)
        self.draw_button(self.height / 2, 250, 300, 50, "Easy")       # A*
        self.draw_button(self.height / 2, 350, 300, 50, "Medium")     # Monte Carlo (MCTS)
        self.draw_button(self.height / 2, 450, 300, 50, "Hard")       # Alpha Beta
        self.draw_button(self.height / 2, 550, 300, 50, "Challenge")  # Decision Tree

    def choose_AI_difficulty(self):
        game_mode = 0
        while True:
            for event in pygame.event.get():
                current_event = event.type
                if current_event == pygame.QUIT: quit()
                elif current_event == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 250 <= mouse_y <= 300:
                        game_mode = 2
                        print("A*")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 350 <= mouse_y <= 400:
                        game_mode = 3
                        print("A* Adversarial")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 450 <= mouse_y <= 500:
                        game_mode = 4
                        print("Alpha Beta")
                    elif (self.width / 2 - 150) <= mouse_x <= (self.width / 2 + 150) and 550 <= mouse_y <= 600:
                        game_mode = 5
                        print("MCTS")
                pygame.display.flip()
                if game_mode != 0:
                    return game_mode

    def draw_board(self):
        """Desenha o tabuleiro do jogo"""
        
        self.screen.fill(s.BACKGROUND_COLOR)

        shadow_coordinates = (2 * self.pixels - 10, self.pixels - 10, self.columns * self.pixels + 24, self.rows * self.pixels + 24)
        board_coordinates = (2 * self.pixels - 10, self.pixels - 10, self.columns * self.pixels + 20, self.rows * self.pixels + 20)
        pygame.draw.rect(self.screen, s.GRAY, shadow_coordinates, 0, 30)
        pygame.draw.rect(self.screen, s.BOARD_COLOR, board_coordinates, 0, 30)

        # desenha os espaços vazios:
        for col in range(self.columns):
            for row in range(self.rows):
                center_of_circle = (int((col + 5 / 2) * self.pixels), int((row + 3 / 2) * self.pixels))
                pygame.draw.circle(self.screen, s.BACKGROUND_COLOR, center_of_circle, self.rad)
        pygame.display.update()

    def draw_new_piece(self, row: int, col: int, piece: int):
        """Desenha a nova peça a ser colocada"""
        
        center_of_circle = (int(col * self.pixels + self.pixels / 2), self.height - int(row * self.pixels + self.pixels / 2))
        pygame.draw.circle(self.screen, s.PIECES_COLORS[piece], center_of_circle, self.rad)

    def draw_button(self, x: int, y: int, width: int, height: int, text: str):
        """Desenha os botões de opção"""
        
        pygame.draw.rect(self.screen, s.GRAY, (x, y, width, height), 0, 30)
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 25)
        text_surface = font.render(text, True, s.BLACK)
        text_rect = text_surface.get_rect()
        text_rect.center = (x + width / 2, y + height / 2)
        self.screen.blit(text_surface, text_rect)

    def show_winner(self, font: any, turn: int):
        """Exibe o vencedor"""
        
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 50)
        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        winner = ("Jogador " + str(turn) + " venceu!")
        pos = (560 - font.size(winner)[0] // 2, 20)
        self.render_alternating_colors_text(winner, font, colors, pos)
        pygame.display.update()

    def show_draw(self, font: any):
        """Exibe mensagem de empate""" 
        
        font = pygame.font.Font('Connect4-main/fonts/SuperMario256.ttf', 50)
        colors = [s.RED, s.YELLOW, s.BLUE, s.GREEN]
        draw_message = "Jogo empatado!"
        pos = (560 - font.size(draw_message)[0] // 2, 20)   
        self.render_alternating_colors_text(draw_message, font, colors, pos)
        pygame.display.update()

    def quit(self):
        pygame.quit()
        sys.exit()


### style.py

**Função:** Implementa funcionalidades do módulo `style.py`.

**Funções e Classes Presentes:**

### Árvore de Decisão (Iris)
![Árvore de Decisão (Iris)](https://github.com/mtsguerra/Connect4/raw/main/training/data/iris_treepkl.png)

In [ ]:
# CORES
RED = (229, 37, 33)  # Vermelho Mario
GREEN = (67, 176, 71)  # Verde Mario
BLUE = (4, 156, 216)  # Azul Mario
YELLOW = (251, 208, 0)  # Amarelo Mario
BLACK = (0, 0, 0)  # Preto
GRAY = (170, 170, 170)  # Cinza
WHITE = (255, 255, 255)  # Branco

BOARD_COLOR = (0,50,245) # Azul tabuleiro
BACKGROUND_COLOR = (155, 225, 255)  # Azul bebê
PLAYER_COLOR = (245, 0, 0)  # Vermelho peça
SECOND_PLAYER_COLOR = (255, 215, 0)  # Amarelo peça

PIECES_COLORS = [WHITE, RED, YELLOW]
FIRST_PLAYER_PIECE = 1  # Valor para o primeiro jogador
SECOND_PLAYER_PIECE = 2  # Valor para o segundo jogador

# Constantes da matriz de dados
ROWS = 6
COLUMNS = 7

# Constantes do tabuleiro
SQUARE_SIZE = 100
RADIUS_PIECE = 45 

# Constantes da tela do jogo
WIDTH = (4 + COLUMNS) * SQUARE_SIZE  # largura da tela = tabuleiro + 2 colunas vazias na direita e esquerda
HEIGHT = (2 + ROWS) * SQUARE_SIZE  # altura da tela = tabuleiro + 1 linha vazia em cima e embaixo


### analyse_dataset.py

O script `analyse_dataset.py` é responsável por carregar e analisar dados de partidas do Connect 4.

#### Funções presentes:

- **`load_dataset(dataset_path)`**
  - Carrega o dataset a partir de um arquivo CSV usando pandas.
  - Mostra o número de amostras e o formato dos dados.

- **`analyze_move_distribution(df)`**
  - Conta a frequência de jogadas em cada coluna do tabuleiro.
  - Calcula e exibe a porcentagem de cada jogada.
  - Gera um gráfico de barras e o salva como `move_distribution.png` na pasta `data/`.


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter

def load_dataset(dataset_path):
    """Carrega o dataset a partir do arquivo CSV"""
    
    print(f"Carregando dataset de {dataset_path}...")
    df = pd.read_csv(dataset_path)
    print(f"Shape do dataset: {df.shape}")
    print(f"Número de amostras: {len(df)}")
    return df

def analyze_move_distribution(df):
    """Analisa e organiza a distribuição dos movimentos no dataset"""
    
    # Conta a ocorrência de cada movimento
    move_counts = Counter(df['move'])
    total_moves = len(df)
    
    # Calcula a porcentagem de cada movimento
    move_percentages = {move: count/total_moves*100 for move, count in move_counts.items()}
    
    print("\nDistribuição dos Movimentos:")
    for move in sorted(move_counts.keys()):
        print(f"Coluna {move}: {move_counts[move]} movimentos ({move_percentages[move]:.2f}%)")
    
    # Cria o gráfico de barras da distribuição dos movimentos
    plt.figure(figsize=(10, 6))
    moves = sorted(move_counts.keys())
    counts = [move_counts[move] for move in moves]
    
    plt.bar(moves, counts)
    plt.title("Distribution of Moves in Dataset")
    plt.xlabel("Column")
    plt.ylabel("Count")
    plt.xticks(moves)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Salva o gráfico na pasta data
    output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "move_distribution.png"))
    plt.close()
    print(f"Gráfico salvo em: {os.path.join(output_dir, 'move_distribution.png')}")

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="Analisar distribuição dos movimentos no Connect4")
    parser.add_argument('--dataset', type=str, help='Caminho para o arquivo CSV do dataset')
    args = parser.parse_args()
    
    if not args.dataset or not os.path.exists(args.dataset):
        print("Por favor, forneça um caminho válido para o dataset usando --dataset")
    else:
        df = load_dataset(args.dataset)
        analyze_move_distribution(df)


### decision_tree.py

**Função:** Implementa funcionalidades do módulo `decision_tree.py`.

**Funções e Classes Presentes:**
- **Função `load_and_split_data()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `evaluate_model()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `train_connect4_model()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `train_iris_model()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `__init__()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `__init__()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `fit()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `predict()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_predict_sample()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_build_tree()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_find_best_feature()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_entropy()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_information_gain()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `save()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `load()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `print_tree()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `_print_node()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Classe `Node`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.
- **Classe `ID3Tree`**: Estrutura que encapsula comportamentos e dados relevantes para o módulo.

### Distribuição dos Movimentos - 300 jogos, 1s
![Distribuição dos Movimentos - 300 jogos, 1s](https://github.com/mtsguerra/Connect4/raw/main/training/data/move_300games1sec_distribution.png)

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import time
import pickle
from collections import Counter
from math import log

# Adiciona o diretório raiz do projeto ao path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

class Node:
    
    def __init__(self):
        self.feature = None      
        self.value = None        
        self.children = {}       
        self.is_leaf = False     
        self.prediction = None   
        self.depth = 0     

class ID3Tree:
    """Implementação mais simples da árvore de decisão ID3"""
    
    def __init__(self, max_depth=10):
        """Inicializa a árvore com profundidade máxima flexível"""
        
        self.root = None
        self.max_depth = max_depth
    
    def fit(self, X, y):
        """Constrói a árvore de decisão a partir dos dados do treino"""
        
        print("Treinando árvore de decisão ID3...")
        start_time = time.time()
        
        X = np.array(X)
        y = np.array(y)
        
        features = list(range(X.shape[1]))
        
        # Constrói a árvore
        self.root = self._build_tree(X, y, features, depth=0)
        
        print(f"Treinamento concluído em {time.time() - start_time:.2f} segundos")
        return self
    
    def predict(self, X):
        """Prevê classes para amostras em X"""
        
        X = np.array(X)
        return np.array([self._predict_sample(x, self.root) for x in X])
    
    def _predict_sample(self, x, node):
        """Prevê a classe para uma única amostra"""
        
        if node.is_leaf:
            return node.prediction
        
        feature_value = x[node.feature]
        
        if feature_value not in node.children:
            predictions = [child.prediction for child in node.children.values() if child.is_leaf]
            if not predictions:
                return list(node.children.values())[0].prediction
            else:
                return Counter(predictions).most_common(1)[0][0]
        
        return self._predict_sample(x, node.children[feature_value])
    
    def _build_tree(self, X, y, features, depth):
        """Constrói a árvore de decisão com recursão"""
        
        node = Node()
        node.depth = depth
        
        # Se todas amostras têm a mesma classe, cria nó folha
        if len(np.unique(y)) == 1:
            node.is_leaf = True
            node.prediction = y[0]
            return node
        
        # Se acabar os atributos ou atingir profundidade máxima, cria uma folha
        if len(features) == 0 or depth >= self.max_depth:
            node.is_leaf = True
            # Usa classe mais comum
            node.prediction = Counter(y).most_common(1)[0][0]
            return node
        
        # Encontra o melhor atributo para dividir, com o maior ganho de informação
        best_feature = self._find_best_feature(X, y, features)
        node.feature = best_feature
        
        # Obtém valores únicos do melhor atributo
        unique_values = np.unique(X[:, best_feature])
        
        # Cria filhos para cada valor
        new_features = features.copy()
        new_features.remove(best_feature)
        
        # Checa se há amostras suficientes para dividir
        if len(unique_values) <= 1:
            node.is_leaf = True
            node.prediction = Counter(y).most_common(1)[0][0]
            return node
        
        # Cria nós filhos
        for value in unique_values:
            # Seleciona amostras com esse valor
            indices = np.where(X[:, best_feature] == value)[0]
            
            if len(indices) == 0:
                continue
            
            # Cria nó filho
            child = self._build_tree(X[indices], y[indices], new_features, depth + 1)
            node.children[value] = child
            
        # Se não criou filhos, cria folha em último caso
        if not node.children:
            node.is_leaf = True
            node.prediction = Counter(y).most_common(1)[0][0]
            
        return node
    
    def _find_best_feature(self, X, y, features):
        """Encontra o atributo com o maior ganho de informação"""
        
        best_gain = -1
        best_feature = None
        base_entropy = self._entropy(y)
        
        for feature in features:
            if len(np.unique(X[:, feature])) <= 1:
                continue
            gain = self._information_gain(X, y, feature, base_entropy)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
        if best_feature is None and features:
            best_feature = features[0]
            
        return best_feature
    
    def _entropy(self, y):
        """Calcula a entropia da distribuição de classes"""
        
        counts = Counter(y)
        total = len(y)
        entropy = 0
        
        for count in counts.values():
            p = count / total
            entropy -= p * log(p, 2)
            
        return entropy
    
    def _information_gain(self, X, y, feature, base_entropy):
        """Calcula o ganho de informação de um atributo"""
        
        values = X[:, feature]
        unique_values = np.unique(values)
        
        # Calcula entropia ponderada
        weighted_entropy = 0
        total_samples = len(y)
        
        for value in unique_values:
            # Seleciona amostras com esse valor
            indices = np.where(values == value)[0]
            subset_y = y[indices]
            
            # Peso e entropia
            weight = len(subset_y) / total_samples
            weighted_entropy += weight * self._entropy(subset_y)
        
        # Ganho de informação
        return base_entropy - weighted_entropy
    
    def save(self, filename):
        """Salva o modelo em arquivo"""
        
        directory = os.path.dirname(filename)
        if directory:
            os.makedirs(directory, exist_ok=True)
            
        with open(filename, 'wb') as f:
            pickle.dump(self, f)
        print(f"Modelo salvo em {filename}")
    
    @staticmethod
    def load(filename):
        """Carrega modelo de arquivo"""
        
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"Modelo carregado de {filename}")
        return model
    
    def print_tree(self, max_depth=3):
        """Imprime uma representação textual da árvore"""
        
        if self.root is None:
            print("Árvore ainda não construída")
            return
        
        self._print_node(self.root, "", True, max_depth)
    
    def _print_node(self, node, indent, is_last, max_depth=None):
        """Imprime um nó e seus filhos"""
        
        if max_depth is not None and node.depth > max_depth:
            return
            
        if node.is_leaf:
            print(f"{indent}{'└── ' if is_last else '├── '}Predição: {node.prediction}")
        else:
            print(f"{indent}{'└── ' if is_last else '├── '}Atributo: {node.feature}")
            
            child_keys = list(node.children.keys())
            for i, value in enumerate(child_keys):
                child = node.children[value]
                is_last_child = (i == len(child_keys) - 1)
                new_indent = indent + ("    " if is_last else "│   ")
                print(f"{new_indent}{'└── ' if is_last_child else '├── '}Valor: {value}")
                
                self._print_node(child, new_indent + "    ", is_last_child, max_depth)

def load_and_split_data(file_path, test_size=0.2, random_seed=42):
    """Carrega os dados do CSV e divide em treino/teste"""
    
    print(f"Carregando dataset de {file_path}...")
    
    # Checa se o arquivo existe
    if not os.path.exists(file_path):
        print(f"Erro: Arquivo {file_path} não encontrado")
        return None, None, None, None
    
    # Carrega os dados
    df = pd.read_csv(file_path)
    print(f"Shape do dataset: {df.shape}")
    
    # Separa atributos e alvo
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values
    
    # Semente para reprodutibilidade
    np.random.seed(random_seed)
    
    # Gera índices para split treino/teste
    indices = np.random.permutation(len(X))
    test_size_int = int(test_size * len(X))
    test_indices = indices[:test_size_int]
    train_indices = indices[test_size_int:]
    
    # Split
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    print(f"Treino: {len(X_train)} amostras")
    print(f"Teste: {len(X_test)} amostras")
    
    return X_train, y_train, X_test, y_test


def evaluate_model(y_true, y_pred):
    """Calcula e imprime métricas de desempenho do modelo"""
    
    # Faz cópia dos arrays para manter os originais
    y_true_clean = np.array(y_true).copy()
    y_pred_clean = np.array(y_pred).copy()
    
    # Substitui valores None nas predições por um valor especial (-1)
    none_indices = [i for i, val in enumerate(y_pred_clean) if val is None]
    if none_indices:
        print(f"Aviso: Encontrado(s) {len(none_indices)} predições None. Substituindo por valor especial para avaliação.")
        for i in none_indices:
            y_pred_clean[i] = -1
    
    # Calcula a acurácia apenas para predições válidas
    valid_indices = [i for i, val in enumerate(y_pred) if val is not None]
    
    if len(valid_indices) > 0:
        accuracy = np.mean(y_true_clean[valid_indices] == y_pred_clean[valid_indices]) * 100
        print(f"Acurácia: {accuracy:.2f}% (excluindo predições None)")
        
        # Calcula a acurácia geral contando None como erro
        overall_accuracy = np.mean(y_true == y_pred) * 100
        print(f"Acurácia geral: {overall_accuracy:.2f}% (contando None como erro)")
    else:
        print("Aviso: Todas as predições são None!")
        accuracy = 0
    
    # Distribuição das classes previstas
    class_dist = {}
    for pred in y_pred:
        pred_key = str(pred)  # Converte para string para lidar com None
        if pred_key not in class_dist:
            class_dist[pred_key] = 0
        class_dist[pred_key] += 1
    
    print("\nDistribuição das classes previstas:")
    for col, count in sorted(class_dist.items()):
        print(f"Coluna {col}: {count} ({count/len(y_pred)*100:.2f}%)")
    
    # Todas classes únicas (exceto None)
    true_classes = set([y for y in y_true if y is not None])
    pred_classes = set([y for y in y_pred if y is not None])
    classes = sorted(true_classes.union(pred_classes))
    
    # Matriz de confusão (excluindo predições None)
    conf_matrix = np.zeros((len(classes), len(classes)), dtype=int)
    
    for i in range(len(y_true_clean)):
        if y_pred_clean[i] == -1:  # Pula predições None
            continue
            
        # Índices nas nossas classes ordenadas
        true_idx = classes.index(y_true_clean[i])
        pred_idx = classes.index(y_pred_clean[i])
        conf_matrix[true_idx, pred_idx] += 1
    
    print("\nMatriz de Confusão (excluindo predições None):")
    header = " " * 10
    for c in classes:
        header += f"Pred {c:<5} "
    print(header)
    
    for i, c in enumerate(classes):
        row = f"True {c:<5} "
        for j in range(len(classes)):
            row += f"{conf_matrix[i, j]:<10} "
        print(row)
    
    # Métricas por classe
    for c in classes:
        c_idx = classes.index(c)
        true_positives = conf_matrix[c_idx, c_idx]
        all_predicted = np.sum(conf_matrix[:, c_idx])
        all_actual = np.sum(conf_matrix[c_idx, :])
        
        precision = true_positives / all_predicted if all_predicted > 0 else 0
        recall = true_positives / all_actual if all_actual > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"\nClasse {c}:")
        print(f"  Precisão: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-score: {f1:.4f}")
    
    return accuracy, conf_matrix


def train_connect4_model(data_path=None, max_depth=10, save_path=None):
    """Treina a decision_tree"""
    
    # Define caminhos padrão se não fornecido
    if data_path is None:
        data_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data", "connect4_dataset_mixed.csv")
    
    if save_path is None:
        save_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data", "connect4_tree_model_mixed.pkl")
    
    # Carrega e divide o dataset
    X_train, y_train, X_test, y_test = load_and_split_data(data_path, test_size=0.2)
    
    if X_train is None:
        print("Erro ao carregar dados. Encerrando.")
        return None
    
    # Treina o modelo
    model = ID3Tree(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    # Avalia no treino
    print("\nAvaliando no treino:")
    y_train_pred = model.predict(X_train)
    train_acc, _ = evaluate_model(y_train, y_train_pred)
    
    # Avalia no teste
    print("\nAvaliando no teste:")
    y_test_pred = model.predict(X_test)
    test_acc, _ = evaluate_model(y_test, y_test_pred)
    
    # Salva modelo
    if save_path:
        model.save(save_path)
    
    # Imprime estrutura da árvore (até 3 níveis)
    print("\nEstrutura da Árvore (até 3 níveis):")
    model.print_tree(max_depth=3)
    
    return model

def train_iris_model(iris_path=None):
    """Treina a decision_tree utilizando a Iris.csv"""
    
    # Caminho padrão, se não fornecido
    if iris_path is None:
        iris_path = os.path.join(
            os.path.dirname(os.path.abspath(__file__)), 
            "iris.csv"
        )
        
    print(f"Carregando dataset iris de {iris_path}...")
    
    # Checa se o arquivo existe
    if not os.path.exists(iris_path):
        print(f"Erro: Dataset iris não encontrado em {iris_path}")
        return None
        
    # Carrega dados
    try:
        df = pd.read_csv(iris_path)
        print(f"Dataset iris carregado com shape: {df.shape}")
        
        # Extrai atributos
        feature_cols = [col for col in df.columns if col.lower() not in ['id', 'class']]
        X = df[feature_cols].values
        
        class_mapping = {
            'Iris-setosa': 0,
            'Iris-versicolor': 1,
            'Iris-virginica': 2
        }
        
        # Coluna de classe
        class_col = [col for col in df.columns if 'class' in col.lower()][0]
        y = df[class_col].map(class_mapping).values
        
        # Checa se o mapeamento funcionou
        if np.any(pd.isna(y)):
            print("Aviso: Alguns rótulos de classe não foram mapeados.")
            print("Classes únicas no dataset:", df[class_col].unique())
            # Tenta corrigir valores não mapeados
            unmapped = df.loc[pd.isna(df[class_col].map(class_mapping)), class_col].unique()
            for i, cls in enumerate(unmapped):
                print(f"Mapeando '{cls}' para {i}")
                y[df[class_col] == cls] = i
        
        # Nomes dos atributos
        feature_names = feature_cols
        print(f"Atributos: {feature_names}")
        print(f"Classes: {list(class_mapping.keys())}")
        
        # Split - sem scikit-learn
        np.random.seed(42)
        indices = np.random.permutation(len(X))
        train_indices = indices[:int(0.8 * len(X))]
        test_indices = indices[int(0.8 * len(X)):]
        
        X_train, y_train = X[train_indices], y[train_indices]
        X_test, y_test = X[test_indices], y[test_indices]
        
        print(f"Amostras de treino: {len(X_train)}")
        print(f"Amostras de teste: {len(X_test)}")
        
        for i in range(X_train.shape[1]):
            values = X_train[:, i]
            thresholds = [
                np.percentile(values, 33),
                np.percentile(values, 66)
            ]
            
            X_train[:, i] = np.digitize(X_train[:, i], thresholds)
            X_test[:, i] = np.digitize(X_test[:, i], thresholds)
        
        # Treina modelo
        model = ID3Tree(max_depth=4)
        model.fit(X_train, y_train)
        
        # Avalia
        print("\nResultados no Dataset Iris:")
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        train_acc = np.mean(train_pred == y_train) * 100
        test_acc = np.mean(test_pred == y_test) * 100
        
        print(f"Acurácia treino: {train_acc:.2f}%")
        print(f"Acurácia teste: {test_acc:.2f}%")
        
        # Classes
        print("\nMapeamento de classes:")
        for cls, idx in class_mapping.items():
            print(f"{idx}: {cls}")
        
        # Imprime árvore
        print("\nÁrvore de Decisão Iris:")
        model.print_tree()
        
        # Salva modelo se diretório existir
        try:
            output_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
            os.makedirs(output_dir, exist_ok=True)
            model.save(os.path.join(output_dir, "iris_tree.pkl"))
        except:
            print("Não foi possível salvar o modelo")
        
        return model
        
    except Exception as e:
        print(f"Erro ao carregar dataset iris: {e}")
        import traceback
        traceback.print_exc()
        return None
    

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description="Treina árvore de decisão ID3 para Connect4")
    parser.add_argument("--data", type=str, help="Caminho para o CSV do dataset Connect4")
    parser.add_argument("--iris-data", type=str, default="training/iris.csv", help="Caminho para o CSV do dataset iris")
    parser.add_argument("--depth", type=int, default=10, help="Profundidade máxima da árvore")
    parser.add_argument("--iris", action="store_true", help="Treina no dataset iris primeiro")
    parser.add_argument("--output", type=str, help="Caminho para salvar o modelo")
    
    args = parser.parse_args()
    
    # Treina no iris se solicitado
    if args.iris:
        print("=== Treinando no Dataset Iris ===")
        train_iris_model(iris_path=args.iris_data)
        print("\n")
    
    # Treina no Connect4
    print("=== Treinando no Dataset Connect4 ===")
    train_connect4_model(
        data_path=args.data,
        max_depth=args.depth,
        save_path=args.output
    )

### decision_tree_player.py

**Função:** Implementa funcionalidades do módulo `decision_tree_player.py`.

**Funções e Classes Presentes:**
- **Função `board_to_feature_vector()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `decision_tree_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `is_valid_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `fallback_strategy()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_next_open_row()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `is_winning_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import os
import sys
import numpy as np

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from game_structure import style as s
from game_structure import game_engine as game
from training.decision_tree import ID3Tree

def board_to_feature_vector(board):
    """Converte o tabuleiro para um vetor de características"""
    
    return board.flatten()

def decision_tree_move(board):
    """Obtém a melhor jogada a partir do decision_tree"""
    
    model_path = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath(__file__))),
        "training", "data", "connect4_tree_model.pkl"
    )

    # Verifica se o modelo existe
    if not os.path.exists(model_path):
        print("Modelo de árvore de decisão não encontrado. Usando estratégia alternativa.")
        return fallback_strategy(board)

    try:
        model = ID3Tree.load(model_path)
        features = np.array([board_to_feature_vector(board)])
        move = model.predict(features)[0]
        # Garante que o movimento é int e está em um intervalo adequado
        try:
            move = int(move)
        except Exception:
            print(f"Movimento previsto {move} não pôde ser convertido para inteiro. Usando alternativa.")
            return fallback_strategy(board)
        if 0 <= move < board.shape[1] and is_valid_move(board, move):
            return move
        else:
            print(f"Movimento inválido {move} da árvore de decisão. Usando alternativa.")
            return fallback_strategy(board)
    except Exception as e:
        print(f"Erro ao usar o modelo de árvore de decisão: {e}")
        return fallback_strategy(board)

def is_valid_move(board, col):
    if col is None or not isinstance(col, (int, np.integer)):
        return False
    if col < 0 or col >= board.shape[1]:
        return False
    return board[0][col] == 0

def fallback_strategy(board):
    # Tenta a coluna central primeiro, depois as colunas adjacentes
    preferred_cols = [3, 2, 4, 1, 5, 0, 6]
    for col in preferred_cols:
        if is_valid_move(board, col):
            return col
    # Alternativa: primeira coluna disponível
    for col in range(board.shape[1]):
        if is_valid_move(board, col):
            return col
    # Nenhum movimento válido
    return 0  # Em vez de -1, sempre retorna 0 (alternativa segura)

def get_next_open_row(board, col):
    for r in range(board.shape[0] - 1, -1, -1):
        if board[r][col] == 0:
            return r
    return -1

def is_winning_move(board, piece):
    from game_structure import game_engine as game
    return game.winning_move(board, piece)

if __name__ == "__main__":
    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
    move = decision_tree_move(board)
    print(f"A árvore de decisão sugere a coluna {move+1} para o tabuleiro vazio")

### generate_connect4_dataset.py

**Função:** Implementa funcionalidades do módulo `generate_connect4_dataset.py`.

**Funções e Classes Presentes:**
- **Função `board_to_feature_vector()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_mcts_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_alphabeta_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_heuristic_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `get_random_move()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `generate_game()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.
- **Função `generate_dataset()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import os
import sys
import time
import csv
import random
import numpy as np
from multiprocessing import Pool, cpu_count
from typing import List, Tuple

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from game_structure import style as s
from game_structure import game_engine as game
from ai_alg.monte_carlo import Node, monte_carlo
from ai_alg.alpha_beta import alpha_beta
from ai_alg.basic_heuristic import evaluate_best_move

# Cria diretório de dados se não existir
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data")
os.makedirs(DATA_DIR, exist_ok=True)

def board_to_feature_vector(board: np.ndarray) -> List[int]:
    """Converte o tabuleiro em um vetor de características flatten"""
    
    return board.flatten().tolist()

def get_mcts_move(board, turn, search_time=1):
    """retorna o movimento do mcts após esperar o último jogador"""
    
    last_player = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
    root = Node(board=board.copy(), last_player=last_player)
    mc = monte_carlo(root)
    return mc.start(search_time)

def get_alphabeta_move(board, turn):
    """retorna o movimento realizado pelo alpha-beta"""
    
    return alpha_beta(board)

def get_heuristic_move(board, turn):
    """retorna o movimento realizado pelo A*"""
    
    opponent = s.FIRST_PLAYER_PIECE if turn == s.SECOND_PLAYER_PIECE else s.SECOND_PLAYER_PIECE
    return evaluate_best_move(board, turn, opponent)

def get_random_move(board, turn):
    """retorna um movimento aleatório"""
    
    moves = game.available_moves(board)
    if isinstance(moves, int) and moves == -1:
        return None
    return random.choice(moves)

def generate_game(params: Tuple[int, int, float, float]) -> List[List[int]]:
    """Simula partidas utilizando diferentes IAs, misturando-as e selecionando aleatoriamente para focar em aumentar a diversidade e qualidade do dataset formado"""
    
    search_time, seed, mcts_prob, ab_prob = params
    np.random.seed(seed)
    random.seed(seed)
    board = np.zeros((s.ROWS, s.COLUMNS), dtype=int)
    records = []
    turn = s.FIRST_PLAYER_PIECE

    AGENTS = [
        ("MCTS", lambda b, t: get_mcts_move(b, t, search_time)),
        ("AlphaBeta", get_alphabeta_move),
        ("Heurística", get_heuristic_move),
        ("Aleatório", get_random_move)
    ]
    # Probabilidades: [MCTS, AlphaBeta, Heurística, Aleatório]
    agent_probs = [mcts_prob, ab_prob, 1 - mcts_prob - ab_prob - 0.05, 0.05]
    agent_probs = [max(0, p) for p in agent_probs]
    sum_probs = sum(agent_probs)
    agent_probs = [p / sum_probs for p in agent_probs]

    while True:
        state = board_to_feature_vector(board)
        agent_idx = np.random.choice(len(AGENTS), p=agent_probs)
        agent_name, agent_func = AGENTS[agent_idx]
        move = agent_func(board, turn)
        # Se a jogada for inválida, tenta aleatório
        moves = game.available_moves(board)
        if (move is None or not (move in moves)):
            move = get_random_move(board, turn)
        records.append(state + [move])
        row = game.get_next_open_row(board, move)
        game.drop_piece(board, row, move, turn)
        if game.winning_move(board, turn) or game.is_game_tied(board):
            break
        turn = s.SECOND_PLAYER_PIECE if turn == s.FIRST_PLAYER_PIECE else s.FIRST_PLAYER_PIECE
    return records

def generate_dataset(n_games: int = 750, search_time: int = 2, out_file: str = "connect4_dataset_mixed.csv", mcts_prob: float = 0.45, ab_prob: float = 0.4) -> None:
    """Gera o dataset dos estados dos jogos e jogadas realizadas pelas múltiplas escolhas realizadas entre as IAs"""
    
    path = os.path.join(DATA_DIR, out_file)
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        header = [f"cell_{i}" for i in range(s.ROWS * s.COLUMNS)] + ["move"]
        writer.writerow(header)

    n_processes = min(cpu_count(), 8)
    pool = Pool(processes=n_processes)
    print(f"Iniciando geração de dataset misto com {n_processes} processos...")

    tasks = [(search_time, i, mcts_prob, ab_prob) for i in range(n_games)]
    completed = 0
    start_time = time.time()
    batch_size = min(10, max(1, n_games // 10))
    for batch_results in pool.imap_unordered(generate_game, tasks, chunksize=batch_size):
        with open(path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerows(batch_results)
        completed += 1
        if completed % batch_size == 0 or completed == n_games:
            elapsed = time.time() - start_time
            eta = (elapsed / completed) * (n_games - completed) if completed > 0 else 0
            print(f"[{time.strftime('%H:%M:%S')}] {completed}/{n_games} partidas geradas "
                  f"({completed/n_games*100:.1f}%) - ETA: {eta/60:.1f} min")
    pool.close()
    pool.join()
    print(f"Dataset salvo em: {path}")
    print(f"Tempo total: {(time.time() - start_time)/60:.2f} minutos")

if __name__ == "__main__":
    print("Iniciando a geração do dataset forte e diverso de Connect4...")
    generate_dataset(
        n_games=750,
        search_time=2,
        out_file="connect4_dataset_mixed.csv",
        mcts_prob=0.45,
        ab_prob=0.45
    )
    print("\nGeração do dataset finalizada.")


### run_connect4_pipeline.py

**Função:** Implementa funcionalidades do módulo `run_connect4_pipeline.py`.

**Funções e Classes Presentes:**
- **Função `run_full_pipeline()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import os
import sys
import time

# Adiciona o diretório raiz do projeto ao path
sys.path.append(os.path.dirname(os.path.abspath(__file__)))

def run_full_pipeline():
    """Executa o pipeline completo do Connect4: treina o modelo, testa o modelo"""
    
    start_time = time.time()
    
    # Como o dataset já foi gerado em outro programa este é apenas para treinar
    
    print("\n=== ETAPA 1: Treinar Modelo de Árvore de Decisão ID3 ===")
    from decision_tree import train_iris_model, train_connect4_model
    
    # Treina no dataset iris como aquecimento
    print("\nTreinando no dataset iris (aquecimento):")
    train_iris_model(iris_path="Connect4-main/training/iris.csv")
    
    # Treina no dataset Connect4
    print("\nTreinando no dataset Connect4:")
    connect4_model = train_connect4_model(
        data_path="Connect4-main/training/data/connect4_dataset_mixed.csv",
        max_depth=10,
        save_path="Connect4-main/training/data/connect4_tree_model_mixed.pkl"
    )
    
    print("\n=== ETAPA 2: Comparar Jogadores de IA ===")
    print("\nNota: Para comparar os jogadores de IA (incluindo Árvore de Decisão, MCTS, Heurística, Alpha Beta), execute:")
    print("python Connect4-main/training/run_ai_vs_ai.py --games 10")
    
    total_time = time.time() - start_time
    print(f"\nPipeline completo executado em {total_time/60:.2f} minutos")

if __name__ == "__main__":
    run_full_pipeline()


### view_tree.py

**Função:** Implementa funcionalidades do módulo `view_tree.py`.

**Funções e Classes Presentes:**
- **Função `adicionar_nos_e_arestas()`**: Responsável por uma etapa específica do fluxo de jogo, IA ou manipulação de dados.

In [ ]:
import pickle
from graphviz import Digraph

# Carrega o modelo de árvore de decisão treinado
with open("Connect4-main/training/data/connect4_tree_model_300games.pkl", "rb") as f:
    model = pickle.load(f)

# Normaliza acesso à raiz da árvore
root = getattr(model, "root", model)

# Cria o grafo Graphviz para visualizar a árvore
dot = Digraph(comment="Árvore de Decisão Connect4")
contador_nos = 0

# Configuração para o grafo crescer de cima para baixo
dot.attr(rankdir="TB")

def adicionar_nos_e_arestas(no, id_pai=None, rotulo_aresta=""):
    """Adiciona recursivamente nós e arestas no grafo para cada nó da árvore"""
    
    global contador_nos
    id_no = f"n{contador_nos}"
    contador_nos += 1

    # Se for folha, destaque e mostre previsão
    if no.is_leaf:
        label = f"Previsão: {no.prediction}"
        dot.node(id_no, label, shape="box", style="filled", color="lightblue")
    else:
        label = f"Pergunta: {no.feature}"
        dot.node(id_no, label, shape="ellipse")

    # Conecta ao nó pai, se existir
    if id_pai is not None:
        dot.edge(id_pai, id_no, label=str(rotulo_aresta))

    # Se não for folha, processa filhos
    if not no.is_leaf:
        for valor, filho in no.children.items():
            adicionar_nos_e_arestas(filho, id_no, rotulo_aresta=valor)

# Começa a recursão pela raiz
adicionar_nos_e_arestas(root)

# Renderiza a árvore para arquivo PNG
saida = dot.render("connect4_tree", format="png", cleanup=False)
print("Árvore de decisão renderizada como 'connect4_tree.png'")



## Considerações Finais

Este projeto explorou diferentes abordagens para construir uma Inteligência Artificial capaz de jogar Connect 4 de maneira competitiva, com destaque para o uso de **Monte Carlo Tree Search (MCTS)** e **Árvores de Decisão (ID3)**, além da análise dos resultados obtidos por cada técnica.

### Comparação entre Algoritmos

- **Árvores de Decisão (ID3)**
  - **Vantagens:**  
    - Simplicidade na implementação e na explicação das decisões tomadas pela IA (“explainable AI”).
    - Treinamento rápido e baixo consumo de recursos computacionais.
    - Bom desempenho em situações com padrões claros e regras fixas.
  - **Desvantagens:**  
    - Limitação na generalização: a árvore pode se tornar excessivamente complexa ou superajustada para situações não previstas nos dados de treino.
    - Dificuldade para capturar estratégias avançadas e planejamento em múltiplos turnos (não considera consequências futuras de uma jogada além da decisão imediata).

- **Monte Carlo Tree Search (MCTS)**
  - **Vantagens:**  
    - Capacidade de planejamento em profundidade, simulando diversos cenários futuros a partir de cada jogada.
    - Mais adaptável a situações inéditas, já que avalia possibilidades “on the fly”.
    - Resultados significativamente superiores contra oponentes humanos e outros algoritmos clássicos quando há tempo suficiente para simular.
  - **Desvantagens:**  
    - Exige maior poder computacional, especialmente conforme o tempo de simulação aumenta.
    - Pode tomar decisões subótimas sob restrições de tempo muito curtas.
    - Difícil de explicar em detalhes o motivo de cada decisão (“black box”).

### Comportamento da IA em Situações do Jogo

Durante o treinamento e os testes, observou-se que:
- A IA baseada em árvore de decisão aprende padrões simples, como evitar deixar três peças adversárias alinhadas, mas frequentemente falha em montar armadilhas ou enxergar ameaças em mais de um turno.
- A abordagem MCTS, por sua vez, consegue defender e atacar de maneira bem mais eficiente, sendo capaz de sacrificar jogadas imediatas para buscar vitória futura, bloquear duplas ameaças e até realizar jogadas de sacrifício para vencer.
- No início da partida, ambas as abordagens tendem a jogar nas colunas centrais, mas apenas o MCTS mantém flexibilidade estratégica ao longo do jogo.
- Em situações de “kill move” (jogada de vitória) ou “must block” (bloqueio obrigatório), MCTS apresenta taxa de sucesso próxima de 100%, enquanto a árvore de decisão pode falhar se nunca viu situação semelhante no treino.

### Vantagens e Limitações Gerais

- **Modularidade:** O projeto é altamente modular, permitindo fácil troca de algoritmos, expansão para novas estratégias (como redes neurais) e adaptação a diferentes jogos de tabuleiro.
- **Explainability:** Árvores de decisão permitem visualizar e explicar facilmente por que certas jogadas são escolhidas, mas não conseguem atingir o nível estratégico do MCTS.
- **Desempenho em tempo real:** Se houver restrição de tempo por jogada, pode ser interessante combinar métodos: usar árvore de decisão para movimentos “fáceis/óbvios” e MCTS para decisões críticas.

**Em resumo, a combinação dos métodos aplicados mostra como diferentes técnicas de IA podem se complementar para resolver problemas complexos de jogos, equilibrando entre explicabilidade, desempenho e adaptação estratégica. O projeto Connect 4 se torna, assim, uma ótima base para estudos e avanços em IA aplicada a jogos e tomada de decisão.**
